# Jupyter Notebook for IFAT VCO implementation

This notebook lays out step-by-step the VCO matrix implementation (proposed by [Welday _et al._, 2008](http://www.jneurosci.org/content/31/45/16157.long)) to be implemented on the IFAT system.


First, we'll import the necessary Python packages:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import LineCollection
import math

import vco  # this is the Python library developed for this project - for clarity, I'll use it in
            # this notebook only after I've already explained/developed the functions

%matplotlib inline

### VCO model as a Python class
First, we create a Python class to model the VCO; this is a purely mathematical representation of the VCO. By this, we mean that the VCO is described by variables and parameters that allow us to calculate the activity for a given spatial path, rather than the VCO being an actual unit oscillating in time. In other words, it is represented by a series of shifted sinusoids, instead of a variable-frequency oscillator.

The first three functions in the class are used internally to initialize and print an instance of the VCO model. Each VCO has the following parameters that describe it:
* N - the number of cells in the VCO
* $\rho$ - the slope of speed modulation (relating to the preferred vector magnitude)
* $\theta$ - the preferred vector orientation
* `phz_noise` - a variable controlling the amount of jitter added to the preferred directions of the VCO's cells (we will leave this at 0, meaning that the preferred directions are each $\frac{2\pi}{N}$ greater than the previous cell.

The last function, `cell_activity`, allows us to see the activity of the VCO at a given location, for a given cell within the VCO.

In [3]:
class VCO_model:

    def __init__(self, N, rho, theta, phz_noise=0):
        self.N = N                       # number of cells in VCO
        self.rho = rho                   # slope of speed modulation
        self.theta = theta               # preferred orientation
        self.phz_noise = phz_noise       # amount of noise introduced in phase offset between each of N cells
        self.cellphz = self._add_noise() # length N array of phase offsets for each cell 

    def __repr__(self):
        rs = 'VCO [N={}, (rho, theta)=({}, {:f}), phi_n={}]'
        return rs.format(self.N, self.rho, self.theta, self.phz_noise)

    def _add_noise(self):
        cellphz = np.zeros(self.N)
        phz_int = 2.0 * np.pi / self.N
        valid = False
        while not valid:
            phase = 0
            for i in range(self.N):
                cellphz[i] = phase
                if (i==(self.N-1)):
                    if not((phase > 2*np.pi) or (phase < 2*(np.pi - phz_int))):
                        valid = True
                else:
                    noise = (2 * phz_int * (nprd.random()-0.5)) * self.phz_noise
                    phase = phase + phz_int + noise
        return cellphz
    
    def cell_activity(self, cell, x, y, F_bar=0):
        x_term = self.rho * np.cos(-self.theta) * x
        y_term = self.rho * np.sin(-self.theta) * y
        phz_term = self.cellphz[cell] + np.pi/2.0
        return F_bar + np.exp(1j * (x_term + y_term + phz_term));

The function `cell_activity(cell, x, y, F_bar)` returns the firing rate of the cell at index `cell` $[0,N)$ in the VCO for the path defined by the arrays $x$ and $y$ (it is organized like this, rather than with a M-by-2 `path` variable, so that it can be computed for a whole arena via the `np.meshgrid()` function). 

The math for this is somewhat complicated; it starts with by defining the instantaneous angular frequency of the VCO (the speed with which a spike travels around the VCO, so effectively the spiking frequency of all N VCO neurons) as:

$$ \omega_n(t) = \Omega(t) + \mathbf{d}_n \cdot \mathbf{v}(t)$$

where $\Omega(t)$ is the "shared angular base frequency" (the theta frequency of ~8Hz), $\mathbf{d}_n$ is the preferred direction vector for VCO $n$, and $\mathbf{v}(t)$ is the velocity vector of the animal (equation 11 in the text). The dot product $\mathbf{d}_n \cdot \mathbf{v}(t)$ is what Ralph was referring to in our meeting.


To get from this equation to the equation in `cell_activity()`, we first integrate the equation above to find the instantaneous _phase_ of the VCO:

$$ \phi_n(t) = \Phi(t) + \delta_n(\mathbf{x}(t))$$

where $\Phi(t)$ is, similarly, the shared base phase (from integrating $\Omega$ over the interval $[0,t)$), and $\delta_n$ is the phase offset of a particular VCO - this depends on the animal's location, $\mathbf{x}(t)$; this means we can bring back in the animal position (which makes sense, since we've integrated velocity):

$$ \phi_n(t) = \Phi(t) + \bigg( \big(\phi_n(0) - \Omega(0)\big) + \mathbf{d}_n \cdot \big[\mathbf{x}(t) - \mathbf{x}(0)\big]\bigg) $$

